# Assignment 3: Build a seq2seq model for machine translation.

### Name: [Abhishek Panda]

### Task: Change LSTM model to Bidirectional LSTM Model， translate English to target language and evaluate using Bleu score.

### Due Date: Tuesday, April 19th, 11:59PM

## 0. You will do the following:

1. Read and run the code. Please make sure you have installed keras or tensorflow.Running the script on colab will speed up the training process and also prevent package loading issue. 
2. Complete the code in Section 1.1, you may fill in your data directory.
3. Directly modify the code in Section 3. Change the current LSTM layer to a Bidirectional LSTM Model.
4. Training your model and translate English to Spanish in Section 4.2. You could try translating other languages.
5. Complete the code in Section 5.

### Hint: 

To implement ```Bi-LSTM```, you will need the following code to build the encoder. Do NOT use Bi-LSTM for the decoder. But there are other codes you need to modify to make it work.

In [1]:
# from keras.layers import Bidirectional, Concatenate, LSTM

# encoder_bilstm = Bidirectional(LSTM(4, return_state=True, 
#                                   dropout=0.5, name='encoder_lstm'))
# _, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

# state_h = Concatenate()([forward_h, backward_h])
# state_c = Concatenate()([forward_c, backward_c])

## 1. Data preparation (10 points)

1. Download spanish-english data from http://www.manythings.org/anki/
2. You may try to use other languages.
3. Unzip the .ZIP file.
4. Put the .TXT file (e.g., "deu.txt") in the directory "./Data/".
5. Fill in your data directory in section 1.1.

### 1.1. Load and clean text


In [2]:
import re
import string
from unicodedata import normalize
import numpy
import os 

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs

def clean_data(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return numpy.array(cleaned)

#### Fill the following blanks:

In [3]:
# e.g., filename = 'Data/deu.txt'
filename = "spa.txt"

# e.g., n_train = 20000
n_train = 20000

In [4]:
# load dataset
doc = load_doc(filename)

# split into Language1-Language2 pairs
pairs = to_pairs(doc)

# clean sentences
clean_pairs = clean_data(pairs)[0:n_train, :]

In [5]:
for i in range(3000, 3010):
    print('[' + clean_pairs[i, 0] + '] => [' + clean_pairs[i, 1] + ']')

[youre here] => [estas aqui]
[youre here] => [estais aqui]
[youre late] => [estas retrasado]
[youre lost] => [estas perdido]
[youre mean] => [eres mala]
[youre mean] => [eres mezquino]
[youre mine] => [tu eres mio]
[youre nice] => [eres simpatico]
[youre nuts] => [estas loco]
[youre nuts] => [estas chiflado]


In [6]:
input_texts = clean_pairs[:, 0]
target_texts = ['\t' + text + '\n' for text in clean_pairs[:, 1]]

print('Length of input_texts:  ' + str(input_texts.shape))
print('Length of target_texts: ' + str(input_texts.shape))

Length of input_texts:  (20000,)
Length of target_texts: (20000,)


In [7]:
max_encoder_seq_length = max(len(line) for line in input_texts)
max_decoder_seq_length = max(len(line) for line in target_texts)

print('max length of input  sentences: %d' % (max_encoder_seq_length))
print('max length of target sentences: %d' % (max_decoder_seq_length))

max length of input  sentences: 18
max length of target sentences: 55


**Remark:** To this end, you have two lists of sentences: input_texts and target_texts

## 2. Text processing

### 2.1. Convert texts to sequences

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# encode and pad sequences
def text2sequences(max_len, lines):
    tokenizer = Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(lines)
    seqs = tokenizer.texts_to_sequences(lines)
    seqs_pad = pad_sequences(seqs, maxlen=max_len, padding='post')
    return seqs_pad, tokenizer.word_index


encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length, 
                                                      input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length, 
                                                       target_texts)

print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

shape of encoder_input_seq: (20000, 18)
shape of input_token_index: 27
shape of decoder_input_seq: (20000, 55)
shape of target_token_index: 29


In [9]:
num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

print('num_encoder_tokens: ' + str(num_encoder_tokens))
print('num_decoder_tokens: ' + str(num_decoder_tokens))

num_encoder_tokens: 28
num_decoder_tokens: 30


**Remark:** To this end, the input language and target language texts are converted to 2 matrices. 

- Their number of rows are both n_train.
- Their number of columns are respective max_encoder_seq_length and max_decoder_seq_length.

The followings print a sentence and its representation as a sequence.

In [10]:
target_texts[100]

'\tno puede ser\n'

In [11]:
decoder_input_seq[100, :]

array([ 6,  8,  3,  1, 17, 14,  2, 15,  2,  1,  5,  2, 10,  7,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0])

## 2.2. One-hot encode

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.
- It is represented by a $n\times t \times v$ tensor ($t$ is the number of unique chars) after the one-hot encoding.

In [12]:
from tensorflow.keras.utils import to_categorical

# one hot encode target sequence
def onehot_encode(sequences, max_len, vocab_size):
    n = len(sequences)
    data = numpy.zeros((n, max_len, vocab_size))
    for i in range(n):
        data[i, :, :] = to_categorical(sequences[i], num_classes=vocab_size)
    return data

encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

print(encoder_input_data.shape)
print(decoder_input_data.shape)

(20000, 18, 28)
(20000, 55, 30)


## 3. Build the networks (for training) (20 points)

- In this section, we have already implemented the LSTM model for you. You can run the code and see what the code is doing.  

- You need to change the existing LSTM model to a Bidirectional LSTM model. Just modify the network structrue and do not change the training cell in section 3.4.

- Build encoder, decoder, and connect the two modules to get "model". 

- Fit the model on the bilingual data to train the parameters in the encoder and decoder.



### 3.1. Encoder network

- Input:  one-hot encode of the input language

- Return: 

    -- output (all the hidden states   $h_1, \cdots , h_t$) are always discarded
    
    -- the final hidden state  $h_t$
    
    -- the final conveyor belt $c_t$

In [13]:
from tensorflow.keras.layers import Input, LSTM
from tensorflow.keras.models import Model
from keras.layers import Bidirectional, Concatenate, LSTM

latent_dim = 256

encoder_inputs = Input(shape=(None, num_encoder_tokens), 
                       name='encoder_inputs')


encoder_lstm = Bidirectional(LSTM(latent_dim , return_state=True, 
                    dropout=0.5, name='encoder_lstm'))

_, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_inputs)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

# Build the encoder network model
encoder_model = Model(inputs=encoder_inputs, outputs=[state_h, state_c],name='encoder')

Print a summary and save the encoder network structure to "./encoder.pdf"

In [14]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(encoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=encoder_model, show_shapes=False,
    to_file='encoder.pdf'
)

encoder_model.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     [(None, None, 28)]   0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   [(None, 512), (None, 583680      encoder_inputs[0][0]             
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 512)          0           bidirectional[0][1]              
                                                                 bidirectional[0][3]              
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 512)          0           bidirectional[0][2]        

### 3.2. Decoder network

- Inputs:  

    -- one-hot encode of the target language
    
    -- The initial hidden state $h_t$ 
    
    -- The initial conveyor belt $c_t$ 

- Return: 

    -- output (all the hidden states) $h_1, \cdots , h_t$

    -- the final hidden state  $h_t$ (discarded in the training and used in the prediction)
    
    -- the final conveyor belt $c_t$ (discarded in the training and used in the prediction)

In [15]:
from keras.layers import Input, LSTM, Dense
from keras.models import Model


latent_dim = 512

# inputs of the decoder network
decoder_input_h = Input(shape=(latent_dim,), name='decoder_input_h')
decoder_input_c = Input(shape=(latent_dim,), name='decoder_input_c')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# set the LSTM layer
decoder_lstm = LSTM(latent_dim, return_sequences=True, 
                    return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_input_x, 
                                                      initial_state=[decoder_input_h, decoder_input_c])

# set the dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# build the decoder network model
decoder_model = Model(inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
                      outputs=[decoder_outputs, state_h, state_c],
                      name='decoder')

Print a summary and save the encoder network structure to "./decoder.pdf"

In [16]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(decoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=decoder_model, show_shapes=False,
    to_file='decoder.pdf'
)

decoder_model.summary()

Model: "decoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input_x (InputLayer)    [(None, None, 30)]   0                                            
__________________________________________________________________________________________________
decoder_input_h (InputLayer)    [(None, 512)]        0                                            
__________________________________________________________________________________________________
decoder_input_c (InputLayer)    [(None, 512)]        0                                            
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, None, 512),  1112064     decoder_input_x[0][0]            
                                                                 decoder_input_h[0][0]      

### 3.3. Connect the encoder and decoder

In [17]:
# input layers
encoder_input_x = Input(shape=(None, num_encoder_tokens), name='encoder_input_x')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# connect encoder to decoder
encoder_final_states = encoder_model([encoder_input_x])
decoder_lstm_output, _, _ = decoder_lstm(decoder_input_x, initial_state=encoder_final_states)
decoder_pred = decoder_dense(decoder_lstm_output)

model = Model(inputs=[encoder_input_x, decoder_input_x], 
              outputs=decoder_pred, 
              name='model_training')

In [18]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=model, show_shapes=False,
    to_file='model_training.pdf'
)

model.summary()

Model: "model_training"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input_x (InputLayer)    [(None, None, 28)]   0                                            
__________________________________________________________________________________________________
decoder_input_x (InputLayer)    [(None, None, 30)]   0                                            
__________________________________________________________________________________________________
encoder (Functional)            [(None, 512), (None, 583680      encoder_input_x[0][0]            
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, None, 512),  1112064     decoder_input_x[0][0]            
                                                                 encoder[0][0]       

### 3.4. Fit the model on the bilingual dataset

- encoder_input_data: one-hot encode of the input language

- decoder_input_data: one-hot encode of the input language

- decoder_target_data: labels (left shift of decoder_input_data)

- tune the hyper-parameters

- stop when the validation loss stop decreasing.

In [19]:
print('shape of encoder_input_data' + str(encoder_input_data.shape))
print('shape of decoder_input_data' + str(decoder_input_data.shape))
print('shape of decoder_target_data' + str(decoder_target_data.shape))

shape of encoder_input_data(20000, 18, 28)
shape of decoder_input_data(20000, 55, 30)
shape of decoder_target_data(20000, 55, 30)


In [20]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics = ['accuracy'])

model.fit([encoder_input_data, decoder_input_data],  # training data
          decoder_target_data,                       # labels (left shift of the target sequences)
          batch_size=64, epochs=50, validation_split=0.2)

model.save('seq2seq.h5')

Epoch 1/50
250/250 [==============================] - 10s 18ms/step - loss: 0.9219 - accuracy: 0.7464 - val_loss: 0.8414 - val_accuracy: 0.7445
Epoch 2/50
250/250 [==============================] - 3s 13ms/step - loss: 0.7115 - accuracy: 0.7838 - val_loss: 0.7204 - val_accuracy: 0.7803
Epoch 3/50
250/250 [==============================] - 3s 13ms/step - loss: 0.6698 - accuracy: 0.7963 - val_loss: 0.6769 - val_accuracy: 0.7937
Epoch 4/50
250/250 [==============================] - 3s 13ms/step - loss: 0.6410 - accuracy: 0.8048 - val_loss: 0.6350 - val_accuracy: 0.8047
Epoch 5/50
250/250 [==============================] - 3s 13ms/step - loss: 0.6204 - accuracy: 0.8110 - val_loss: 0.6129 - val_accuracy: 0.8092
Epoch 6/50
250/250 [==============================] - 3s 13ms/step - loss: 0.6014 - accuracy: 0.8168 - val_loss: 0.5890 - val_accuracy: 0.8174
Epoch 7/50
250/250 [==============================] - 3s 13ms/step - loss: 0.5864 - accuracy: 0.8213 - val_loss: 0.5726 - val_accuracy: 0.823

## 4. Make predictions

- In this section, you need to complete section 4.2 to translate English to the target language.


### 4.1. Translate English to Spanish

1. Encoder read a sentence (source language) and output its final states, $h_t$ and $c_t$.
2. Take the [star] sign "\t" and the final state $h_t$ and $c_t$ as input and run the decoder.
3. Get the new states and predicted probability distribution.
4. sample a char from the predicted probability distribution
5. take the sampled char and the new states as input and repeat the process (stop if reach the [stop] sign "\n").

In [21]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [22]:
def decode_sequence(input_seq, temperature = 0.2):
    states_value = encoder_model.predict(input_seq)

    target_seq = numpy.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # this line of code is greedy selection
        # try to use multinomial sampling instead (with temperature)
        # sampled_token_index = numpy.argmax(output_tokens[0, -1, :])

        # Multinomial sampling with temperature
        p = output_tokens[0, -1, :]
        p = numpy.asarray(p).astype('float64')
        p = p ** (1 / temperature)
        p = p / numpy.sum(p)

        next_onehot = numpy.random.multinomial(1,p,1)
        sampled_token_index = numpy.argmax(next_onehot)

        # To handle the 0 index error
        if sampled_token_index == 0:
            break
        
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = numpy.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


In [23]:
for seq_index in range(2100, 2120):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('English:       ', input_texts[seq_index])
    print('Spanish (true): ', target_texts[seq_index][1:-1])
    print('Spanish (pred): ', decoded_sentence[0:-1])

-
English:        hes skinny
Spanish (true):  el esta delgado
Spanish (pred):  el es amagle
-
English:        hes strong
Spanish (true):  el es fuerte
Spanish (pred):  el es amable
-
English:        hes stupid
Spanish (true):  el es estupido
Spanish (pred):  es esta listo
-
English:        hes stupid
Spanish (true):  no le llega agua al tanque
Spanish (pred):  el es a tu
-
English:        hes stupid
Spanish (true):  es un salame
Spanish (pred):  el es amariente
-
English:        help me out
Spanish (true):  ayudame
Spanish (pred):  ayudame
-
English:        help me out
Spanish (true):  ayudame a salir
Spanish (pred):  ayudame
-
English:        help me out
Spanish (true):  echeme la mano
Spanish (pred):  ayudame
-
English:        help me out
Spanish (true):  ayudame a salir
Spanish (pred):  ayudame
-
English:        here i come
Spanish (true):  aqui vengo
Spanish (pred):  aqui estamos
-
English:        here i come
Spanish (true):  ya estoy aqui
Spanish (pred):  aqui estamos
-
English:  

### 4.2. Translate an English sentence to the target language （20 points）

1. Tokenization
2. One-hot encode
3. Translate

In [24]:
input_sentence = 'I love you'

tokenizer = Tokenizer(char_level=True, filters='')
tokenizer.word_index = input_token_index
sequence = tokenizer.texts_to_sequences([input_sentence])

input_sequence = pad_sequences(sequence, maxlen=max_encoder_seq_length, dtype='int32', padding='post')

input_x = onehot_encode(input_sequence,max_encoder_seq_length,num_encoder_tokens )

output_sentence = decode_sequence(input_x)

print('source sentence is: ' + input_sentence)
print('translated sentence is: ' + output_sentence)

source sentence is: I love you
translated sentence is: yo amo a tom



### 5.1. Partition the dataset to training, validation, and test. Build new token index. (10 points)

1. You may try to load more data/lines from text file.
2. Convert text to sequences and build token index using training data.
3. One-hot encode your training and validation text sequences.

In [25]:
cleaned_pairs = clean_data(pairs)[0:100000, :]
input_texts = cleaned_pairs[:, 0]
target_texts = ['\t' + text + '\n' for text in cleaned_pairs[:, 1]]

print('Length of input_texts:  ' + str(input_texts.shape))
print('Length of target_texts: ' + str(input_texts.shape))

Length of input_texts:  (100000,)
Length of target_texts: (100000,)


In [26]:
max_encoder_seq_length = max(len(line) for line in input_texts)
max_decoder_seq_length = max(len(line) for line in target_texts)

print('max length of input  sentences: %d' % (max_encoder_seq_length))
print('max length of target sentences: %d' % (max_decoder_seq_length))

max length of input  sentences: 36
max length of target sentences: 87


In [27]:
encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length, 
                                                      input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length, 
                                                       target_texts)

In [28]:
print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

shape of encoder_input_seq: (100000, 36)
shape of input_token_index: 27
shape of decoder_input_seq: (100000, 87)
shape of target_token_index: 29


In [29]:
num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

In [30]:
num_encoder_tokens

28

In [31]:
num_decoder_tokens

30

In [32]:
encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq, max_decoder_seq_length, num_decoder_tokens)

In [33]:
encoder_input_data.shape

(100000, 36, 28)

In [34]:
decoder_input_data.shape

(100000, 87, 30)

In [35]:
decoder_target_data.shape

(100000, 87, 30)

In [36]:
rand_indices = numpy.random.permutation(100000)
train_rand_indices = rand_indices[0:70000]
valid_rand_indices = rand_indices[70000:85000]
test_rand_indices = rand_indices[85000:100000]


encoder_input_data_train = encoder_input_data[train_rand_indices,:]
encoder_input_data_valid = encoder_input_data[valid_rand_indices,:]
encoder_input_data_test = encoder_input_data[test_rand_indices,:] 

decoder_input_data_train = decoder_input_data[train_rand_indices,:]
decoder_input_data_valid = decoder_input_data[valid_rand_indices,:]
decoder_input_data_test = decoder_input_data[test_rand_indices,:] 


decoder_target_data_train = decoder_target_data[train_rand_indices,:]
decoder_target_data_valid = decoder_target_data[valid_rand_indices,:]
decoder_target_data_test = decoder_target_data[test_rand_indices,:] 

In [37]:
encoder_input_data_train.shape

(70000, 36, 28)

### 5.2 Retrain your previous Bidirectional LSTM model with training and validation data and tune the parameters (learning rate, optimizer, etc) based on validation score. (25 points)

1. Use the model structure in section 3 to train a new model with new training and validation datasets.
2. Based on validation BLEU score or loss to tune parameters.

In [38]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit([encoder_input_data_train, decoder_input_data_train],  # training data
          decoder_target_data_train,                       # labels (left shift of the target sequences)
          batch_size=1028, epochs=50, validation_data=([encoder_input_data_valid, decoder_input_data_valid], decoder_target_data_valid))

model.save('seq2seq_tune.h5')

Epoch 1/50
69/69 [==============================] - 16s 167ms/step - loss: 0.8657 - val_loss: 0.6922
Epoch 2/50
69/69 [==============================] - 10s 150ms/step - loss: 0.7242 - val_loss: 0.6258
Epoch 3/50
69/69 [==============================] - 11s 153ms/step - loss: 0.6864 - val_loss: 0.5736
Epoch 4/50
69/69 [==============================] - 11s 155ms/step - loss: 0.6618 - val_loss: 0.5424
Epoch 5/50
69/69 [==============================] - 11s 157ms/step - loss: 0.6446 - val_loss: 0.5304
Epoch 6/50
69/69 [==============================] - 11s 156ms/step - loss: 0.6311 - val_loss: 0.5060
Epoch 7/50
69/69 [==============================] - 11s 157ms/step - loss: 0.6195 - val_loss: 0.4893
Epoch 8/50
69/69 [==============================] - 11s 157ms/step - loss: 0.6094 - val_loss: 0.4830
Epoch 9/50
69/69 [==============================] - 11s 158ms/step - loss: 0.6007 - val_loss: 0.4699
Epoch 10/50
69/69 [==============================] - 11s 157ms/step - loss: 0.5924 - val_lo

In [39]:
input_test_texts = input_texts[test_rand_indices]
target_texts = numpy.array(target_texts)
target_test_texts = target_texts[test_rand_indices]

predicted_sentences = []

# for seq_index in range(2100, 2120):
for seq_index in range(2120, 2150):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    predicted_sentences.append(decoded_sentence[0:-1])
    print('-')
    print('English:       ', input_test_texts[seq_index])
    print('Spanish (true): ', target_test_texts[seq_index][1:-1])
    print('Spanish (pred): ', decoded_sentence[0:-1])


-
English:        i dont like you anymore
Spanish (true):  ya no me gustas
Spanish (pred):  	a de ymsno nm cadpo
-
English:        stand up
Spanish (true):  de pie
Spanish (pred):  uejo ue rrodo
-
English:        the car didnt move
Spanish (true):  el auto no se movio
Spanish (pred):  er 	t	a 	a co	a o do
-
English:        we got to the station at six
Spanish (true):  llegamos a la estacion a las seis
Spanish (pred):  	as godas e	 er ca	eyta ue cacge
-
English:        i know this is stupid
Spanish (true):  se que esto es estupido
Spanish (pred):  se vme es esno fe
-
English:        i read your report
Spanish (true):  he leido su informe
Spanish (pred):  re o	co	no nm cacge
-
English:        could you give me a lift home
Spanish (true):  me acercas a casa
Spanish (pred):  pau
-
English:        are you saying my life is in danger
Spanish (true):  estas diciendo que estoy en peligro
Spanish (pred):  esnos uesoua ue esno de	o	ua e	 m	 cacge
-
English:        tom lost his wallet
Spanish (tr

### 5.3 Evaluate the BLEU score using the test set. (15 points)

1. Use trained model above to calculate the BLEU score with testing dataset.

In [40]:
target_list = [sentence.split() for sentence in target_test_texts[2120:2150]]
predicted_list = [sentence.split() for sentence in predicted_sentences]

In [41]:
# Compute the BLEU score
from nltk.translate.bleu_score import corpus_bleu

references = target_list
candidates = predicted_list
score = corpus_bleu(references, candidates)
print(score)

1.0935728289669988e-231


C:\Users\abhis\.conda\envs\GPU\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\abhis\.conda\envs\GPU\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\abhis\.conda\envs\GPU\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingF